# Sequential Executors - Class-Based Pattern

## Overview
This notebook demonstrates how to build sequential workflows using **class-based executors** with the Agent Framework. You'll learn how to:
- Create custom `Executor` classes with `@handler` methods
- Use `WorkflowContext` with type annotations for type-safe message passing
- Chain executors sequentially using `add_edge()`
- Send intermediate messages with `ctx.send_message()`
- Yield final outputs with `ctx.yield_output()`
- Observe workflow execution events via streaming

## What This Sample Does
The workflow performs a simple text transformation pipeline:
1. **UpperCaseExecutor** - Converts input text to uppercase and sends it to the next executor
2. **ReverseTextExecutor** - Receives the uppercase text, reverses it, and yields the final output

**Input:** `"hello world"`  
**Output:** `"DLROW OLLEH"` (uppercase, then reversed)

## Key Concepts
- **Executor Classes:** Explicit classes inheriting from `Executor` provide clear structure for complex logic
- **@handler Decorator:** Marks the method that processes incoming messages
- **WorkflowContext[T_Out, T_W_Out]:** Generic types ensure type safety for intermediate (`T_Out`) and final (`T_W_Out`) outputs
- **Sequential Chaining:** `add_edge(A, B)` creates a direct path from executor A to B
- **Event Streaming:** `run_stream()` provides observability into execution flow

## Step 1: Import Required Libraries
Import the Agent Framework components for building and executing workflows.

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio

import os
from dotenv import load_dotenv
from agent_framework import (
    Executor,
    ExecutorCompletedEvent,
    ExecutorInvokedEvent,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    handler,
)
from typing_extensions import Never
# Load environment variables from .env file
load_dotenv('../../.env')


## Step 2: Define the UpperCaseExecutor
This executor converts input text to uppercase and sends it to the next stage.

**Key Points:**
- Inherits from `Executor` base class
- Uses `@handler` to mark the processing method
- `WorkflowContext[str]` indicates this executor outputs `str` to downstream executors
- `ctx.send_message()` passes the transformed text to the next executor

In [ ]:
class UpperCaseExecutor(Executor):
    """An executor that transforms text to uppercase."""

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "upper_case_executor")

    @handler
    async def to_upper_case(self, text: str, ctx: WorkflowContext[str]) -> None:
        """Execute the task by converting text to uppercase."""
        upper_text = text.upper()
        await ctx.send_message(upper_text)

## Step 3: Define the ReverseTextExecutor
This executor reverses the input text and yields the final workflow output.

**Key Points:**
- `WorkflowContext[Never, str]` means:
  - `Never` for intermediate output (doesn't send messages to other executors)
  - `str` for final workflow output type
- `ctx.yield_output()` marks this as the final result
- Terminal executor in the chain

In [ ]:
class ReverseTextExecutor(Executor):
    """An executor that reverses text."""

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "reverse_text_executor")

    @handler
    async def reverse_text(self, text: str, ctx: WorkflowContext[Never, str]) -> None:
        """Execute the task by reversing the text."""
        reversed_text = text[::-1]
        await ctx.yield_output(reversed_text)

## Step 4: Build the Sequential Workflow
Create the workflow graph by connecting executors in sequence.

**Workflow Structure:**
```
Input → UpperCaseExecutor → ReverseTextExecutor → Output
```

**Key Points:**
- `set_start_executor()` defines the entry point
- `add_edge(A, B)` creates a direct connection from A to B
- Sequential execution is guaranteed by the edge order

In [ ]:
async def run_workflow():
    """Build and execute the sequential workflow."""
    # Step 1: Create the executors
    upper_case_executor = UpperCaseExecutor()
    reverse_text_executor = ReverseTextExecutor()

    # Step 2: Build the workflow with sequential edges
    workflow = (
        WorkflowBuilder()
        .set_start_executor(upper_case_executor)
        .add_edge(upper_case_executor, reverse_text_executor)
        .build()
    )

    # Step 3: Run the workflow with streaming and observe events
    input_text = "hello world"
    print(f"Input: {input_text}")
    print("\n=== Workflow Events ===")
    
    async for event in workflow.run_stream(input_text):
        if isinstance(event, ExecutorInvokedEvent):
            print(f"▶ Executor Invoked: {event.executor_id}")
        elif isinstance(event, ExecutorCompletedEvent):
            print(f"✓ Executor Completed: {event.executor_id}")
        elif isinstance(event, WorkflowOutputEvent):
            print(f"\n=== Final Output ===")
            print(f"Result: {event.data}")

## Step 5: Execute the Workflow
Run the workflow and observe the event stream.

In [ ]:
await run_workflow()

## Expected Output
```
Input: hello world

=== Workflow Events ===
▶ Executor Invoked: upper_case_executor
✓ Executor Completed: upper_case_executor
▶ Executor Invoked: reverse_text_executor
✓ Executor Completed: reverse_text_executor

=== Final Output ===
Result: DLROW OLLEH
```

## Key Takeaways

### Class-Based Executor Pattern
✅ **Explicit Structure:** Classes provide clear boundaries for executor logic  
✅ **State Management:** Executors can maintain instance variables across invocations  
✅ **Type Safety:** `WorkflowContext` generic types enforce correct message types  
✅ **Reusability:** Executors can be instantiated multiple times with different configurations

### Sequential Execution Flow
1. **Input** → UpperCaseExecutor receives `"hello world"`
2. **Transform** → Converts to `"HELLO WORLD"`
3. **Send** → `ctx.send_message()` passes to ReverseTextExecutor
4. **Transform** → Reverses to `"DLROW OLLEH"`
5. **Output** → `ctx.yield_output()` completes the workflow

### Event Streaming Benefits
- **Observability:** Track executor invocations and completions in real-time
- **Debugging:** Identify where execution is slow or failing
- **Monitoring:** Build production-grade observability into workflows

### When to Use Class-Based Executors
- Complex logic requiring state management (counters, accumulators)
- Executors with initialization parameters
- Multiple handler methods or helper functions
- Reusable components across different workflows

## Next Steps
- See `sequential_streaming.ipynb` for the **function-based** alternative using `@executor`
- Explore `edge_condition.ipynb` for **conditional routing** patterns
- Check `simple_loop.ipynb` to learn about **iterative workflows**